# It's a Machine and Natural: Generation

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_gum
from collections import defaultdict

seed = 691
all_docs = load_gum(num_articles = 0, seed = seed)
nsamp = min(148, len(all_docs)-1) ## this is the size of the test document base..., so nsamp=125 => test == 125 & train == 25!
test_docs = all_docs[:nsamp]
train_docs = all_docs[nsamp:]
docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
covering_vocab = set([t for d in covering for s in d for t in s])

layers = [[[[row[2] for row in s] for s in d['conllu']] for d in train_docs],
          [[[row[3] for row in s] for s in d['conllu']] for d in train_docs]]
ltypes = ['lem','pos']

model = LM(form2ind={"<sos>": 0, "<pad>": 1}, covering_vocab = covering_vocab)
model.init(m = 10, zeta = 0.01, positional = True, seed = seed)
model.fit(docs, f'GUM-{nsamp}', covering = covering, layers = layers, ltypes = ltypes)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 152212.65it/s]


Training tokenizer...


Fitting:   4%|████▉                                                                                                                       | 4/100 [00:00<00:05, 16.07it/s]


Built a vocabulary of 557 types
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.27it/s]


...

Absorbing pos-layer co-occurrences...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  8.56it/s]


Computing marginal statistics...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200454/200454 [00:00<00:00, 255438.32it/s]


Done.
Model params, tokens, contexts, and % capacity used: 200454 1074 21005 0.889


In [2]:
model.generate()
[doc._form for doc in model._documents]

0it [00:00, ?it/s]


['Add']

In [3]:
model.generate(seed = 691)
[doc._form for doc in model._documents]

['You']

In [4]:
model.generate(seed = 691)
[doc._form for doc in model._documents]

['onions']

In [5]:
model.generate(m = 25, generate_next = 's', seed = 691)
[doc._form for doc in model._documents]

['thirdcupspeppersof!liquid:way']

In [6]:
print(docs[-2][1][:-14])
model.generate(m = 25, docs = [[docs[-2][1][:-14]]], generate_next = 's', seed = 691)
[doc._form for doc in model._documents]

Quinoa is known as the little 
Tokenizing documents...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 420.10it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16008.79it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.86it/s]


['Quinoa is known as the little rice.of']

In [7]:
print(docs[-2][1][:-14])
model.generate(m = 25, docs = [[docs[-2][1][:-14]]], generate_next = 's', seed = 691, resonant = True)
[doc._form for doc in model._documents]

Quinoa is known as the little 
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.68it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15709.00it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.65it/s]


['Quinoa is known as the little rice.of']

In [8]:
from collections import Counter
print(docs[-2][1][:-14])
model.generate(m = 25, docs = [[docs[-2][1][:-14]]], generate_next = 's', seed = 691, resonant = True,
               Td = Counter([model.tokenizer.tokenize(docs[-2][0])]))
[doc._form for doc in model._documents]

Quinoa is known as the little 
Tokenizing documents...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 514.95it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18157.16it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.64it/s]


['Quinoa is known as the little rice. ']

In [9]:
print(tdocs[1][4][:39])  
model.generate(m = 25, docs = [[tdocs[1][4][:39]]], generate_next = 's', seed = 691, resonant = True,
               Td = Counter([t for tdoc in tdocs[1][:4] for t in model.tokenizer.tokenize(tdoc)]))
[doc._form for doc in model._documents]

Additionally, diversity has been shown 
Tokenizing documents...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 440.76it/s]


Indexing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9383.23it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]


['Additionally, diversity has been shown with it,asletsupshcreagiontiontiontionformstiontiontiontiontiontiontion']

In [10]:
print(tdocs[1][4][:39])  
model.generate(m = 75, docs = [[tdocs[1][4][:39]]], generate_next = 'd', seed = 691, resonant = True,
               Td = Counter([t for tdoc in tdocs[1][:4] for t in model.tokenizer.tokenize(tdoc)]))
[doc._form for doc in model._documents]

Additionally, diversity has been shown 
Tokenizing documents...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 464.07it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14266.34it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]


['Additionally, diversity has been shown with it,asletsupshcreagiontiontiontionformstiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontiontion']